In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from path import Path

### Import Data

In [2]:
# Read tornadoes csv into DataFrame
file_path = Path('Raw_Data/1950-2020_torn.csv')
tornadoes_df = pd.read_csv(file_path, index_col=0)
tornadoes_df

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67500,619522,2020,9,1,9/1/2020,18:10:00,3,TX,48,0,...,0.01,30,1,1,1,191,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [3]:
tornadoes_df.columns

Index(['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'f',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')

In [4]:
tornadoes_df.dtypes

om         int64
yr         int64
mo         int64
dy         int64
date      object
time      object
tz         int64
st        object
stf        int64
stn        int64
f          int64
inj        int64
fat        int64
loss     float64
closs    float64
slat     float64
slon     float64
elat     float64
elon     float64
len      float64
wid        int64
ns         int64
sn         int64
sg         int64
f1         int64
f2         int64
f3         int64
f4         int64
fc         int64
dtype: object

In [8]:
tornadoes_df["datetime"] = tornadoes_df.date.str.cat(tornadoes_df.time, sep=' ')

tornadoes_df

NameError: name 'df' is not defined

In [6]:
# convert date and time to datetime dtype
tornadoes_df["datetime"] = pd.to_datetime(tornadoes_df["datetime"])

ParserError: Unknown string format: 10/1/195021:00:00

In [ ]:
# Check dtypes
tornadoes_df.dtypes

In [ ]:
# Rename Columns
tornadoes_df.rename(columns = {"stf":"st_fips",'f':'ef'}, inplace = True)

In [ ]:
# Rearrange colums
cols = ['yr', 'mo', 'dy', 'datetime', 'st', 'st_fips', 'f1', 'f2', 'f3', 'f4', 
        'ef', 'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len', 'wid' ]

tornadoes_df[cols]

In [ ]:
# Sort DF by Year then State
tornadoes_df.sort_values(by=['yr','mo','st'])

### Clean the tor_EF dataframe

### Clean the whole dataset

In [ ]:
# Filter out any bad data (ie 0<=EF<=5)
tornadoes_clean = tornadoes_df[ (tornadoes_df['ef'] >= 0) & ( tornadoes_df["ef"] <= 5)]
tornadoes_clean

In [ ]:
# Filter for Lower 48 (exclude PR, VI, HI, AK)
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'PR']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'VI']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'HI']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'AK']
tornadoes_clean

In [ ]:
# Investigate closs
tornadoes_clean["closs"].value_counts()

In [ ]:
# Replace loss and closs 0 with NaN ( 0 means no data avaliable NOT $0 in loss)
tornadoes_clean['closs']=tornadoes_clean['closs'].replace(0, np.nan)
tornadoes_clean['loss']=tornadoes_clean['loss'].replace(0, np.nan)
tornadoes_clean

In [ ]:
# Starting in 2016 the loss data is in whole dollar amounts need to convert rows with years prior to 2016 to whole dollar
# original loss and closs were in millions of dollars
# Multiply loss and closs by $1,000,000 to get whole number values
loss = tornadoes_clean.loc[tornadoes_clean["yr"]<2016]
loss

In [ ]:
loss["loss"] = loss["loss"]*1000000
loss["closs"] = loss["closs"]*1000000
loss

In [ ]:
# drop all rows in tor_EF between 2008-2015
tornadoes_clean_2016_2020 = tornadoes_clean[tornadoes_clean["yr"] >= 2016]
tornadoes_clean_2016_2020

In [ ]:
tornadoes_clean = loss.append(tornadoes_clean_2016_2020)
tornadoes_clean

In [ ]:
tornadoes_clean.columns

### Filter cleaned data for for years needed 2008-2020 (EF scale tornado ratings)

In [ ]:
# Delete columns we dont need
tornadoes_df_clean = tornadoes_clean.drop(["om","tz","stn", 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'], axis =1)

In [ ]:
# 1950-2020
tornadoes_df_clean.to_csv("Cleaned_Data/1950-2020_tornadoes_cleaned.csv")

In [ ]:
# Filter for years 1950-2007 --> F Scale
tornadoes_F = tornadoes_df_clean[tornadoes_df['yr'] < 2008]
tornadoes_F

In [ ]:
# Export Cleaned Data to CSV
tornadoes_F.to_csv("Cleaned_Data/1950-2007_tornadoes_F_cleaned.csv")

In [ ]:
# Filter for years 2008-2020 --> EF Scale
tornadoes_EF = tornadoes_df_clean[tornadoes_df['yr'] >= 2008]
tornadoes_EF

In [ ]:
# Export Cleaned Data to CSV
tornadoes_EF.to_csv("Cleaned_Data/2008-2020_tornadoes_EF_cleaned.csv")

In [ ]:
# Summary Stats on Data
tornadoes_EF[["ef","inj","fat","loss","closs","len","wid"]].describe()